# Overview
Simplifies and converts data to urdf.  
Note that the notebook initially was in the data folder, so copy it or fix paths laters.

In [ ]:
import os, shutil, pymeshlab, glob
from tqdm.notebook import tqdm 

SRC_ROOT = 'models'
DST_ROOT = 'simplified_models'
TARGET_PERC = 0.05           # keep 5 % of the faces

for i in tqdm(range(88)):
    folder = f'{i:03d}'
    src_dir = os.path.join(SRC_ROOT, folder)
    dst_dir = os.path.join(DST_ROOT, folder)
    os.makedirs(dst_dir, exist_ok=True)

    src_obj = os.path.join(src_dir, 'textured.obj')
    dst_obj = os.path.join(dst_dir, 'textured_simplified.obj')
    if not os.path.isfile(src_obj):
        continue

    ms = pymeshlab.MeshSet()
    ms.load_new_mesh(src_obj)
    m = ms.current_mesh()

    try:
        if m.has_wedge_tex_coord() or m.has_vertex_tex_coord():
            # make the UVs consistent first
            if not m.has_wedge_tex_coord():
                ms.apply_filter('convert_pervertex_uv_into_perwedge_uv')

            ms.apply_filter('meshing_decimation_quadric_edge_collapse_with_texture',
                            targetperc=TARGET_PERC)              # :contentReference[oaicite:2]{index=2}
        else:
            ms.apply_filter('meshing_decimation_quadric_edge_collapse',
                            targetperc=TARGET_PERC)
    except pymeshlab.PyMeshLabException as err:
        # fall back to geometry‑only simplification
        print('Texture‑aware decimator failed, trying without texture:', err)
        ms.apply_filter('meshing_decimation_quadric_edge_collapse',
                        targetperc=TARGET_PERC)

    ms.save_current_mesh(dst_obj, save_textures=True)

    # copy any referenced images so the .mtl keeps working
    mtl_out = os.path.splitext(dst_obj)[0] + '.mtl'
    if os.path.isfile(mtl_out):
        with open(mtl_out) as fh:
            for line in fh:
                if line.lower().startswith(('map_kd', 'map_ks', 'map_bump', 'map_d')):
                    tex = os.path.basename(line.split()[-1])
                    if os.path.isfile(os.path.join(src_dir, tex)):
                        shutil.copy2(os.path.join(src_dir, tex),
                                     os.path.join(dst_dir, tex))

    print(f'✔  {src_obj} → {dst_obj}')


Texture‑aware decimator failed, trying without texture: Failed to apply filter: meshing_decimation_quadric_edge_collapse_with_texture
Details: Mesh has some inconsistent tex coordinates (some faces without texture)
✔  models/075/textured.obj → simplified_models/075/textured_simplified.obj
Texture‑aware decimator failed, trying without texture: Failed to apply filter: meshing_decimation_quadric_edge_collapse_with_texture
Details: Mesh has some inconsistent tex coordinates (some faces without texture)
✔  models/076/textured.obj → simplified_models/076/textured_simplified.obj
Texture‑aware decimator failed, trying without texture: Failed to apply filter: meshing_decimation_quadric_edge_collapse_with_texture
Details: Mesh has some inconsistent tex coordinates (some faces without texture)
✔  models/077/textured.obj → simplified_models/077/textured_simplified.obj
Texture‑aware decimator failed, trying without texture: Failed to apply filter: meshing_decimation_quadric_edge_collapse_with_text

In [ ]:
/home/dgrachev/Vision-Language-Grasping/assets/simplified_objects/001/textured_simplified.obj

In [8]:
import os, shutil
from object2urdf import ObjectUrdfBuilder

SRC_ROOT = '../assets/simplified_objects'
PROTO = os.path.join(SRC_ROOT, 'obj_prototype.urdf')  # one master prototype

for i in range(1, 2):
    obj_dir = os.path.join(SRC_ROOT, f'{i:03d}')          
    mesh_name = 'textured_simplified.obj'                 
    mesh_path = os.path.join(obj_dir, mesh_name)

    if not os.path.isfile(mesh_path):
        print(f'Skipping {mesh_path} (missing)')
        continue

    
    dst_proto = os.path.join(obj_dir, 'obj_prototype.urdf')
    if not os.path.isfile(dst_proto):
        shutil.copy2(PROTO, dst_proto)

    # build the URDF
    builder = ObjectUrdfBuilder(obj_dir, urdf_prototype='obj_prototype.urdf')
    # NOTE: pass the path *relative* to obj_dir
    urdf_path = builder.build_urdf(filename=mesh_path,          # relative!
                                   force_overwrite=True,
                                   decompose_concave=True,
                                   force_decompose=False,
                                   center='mass')               # or 'geometric', 'top', …

    print('✔  wrote', urdf_path)


failed to load image
Traceback (most recent call last):
  File "/home/dgrachev/miniconda3/envs/vilg/lib/python3.8/site-packages/trimesh/exchange/obj.py", line 337, in parse_mtl
    file_data = resolver.get(split[1])
  File "/home/dgrachev/miniconda3/envs/vilg/lib/python3.8/site-packages/trimesh/resolvers.py", line 91, in get
    with open(path, 'rb') as f:
FileNotFoundError: [Errno 2] No such file or directory: '/home/dgrachev/Vision-Language-Grasping/assets/simplified_objects/001/texture_map.png'


✔  wrote None


In [6]:
!ls ../assets/

simplified_objects  ur5e  workspace


In [2]:
import shutil
import os

src_folder = "simplified_models"
dst_folder = os.path.join("..", "assets", "simplified_objects", os.path.basename(src_folder))

# Remove destination folder if it already exists
if os.path.exists(dst_folder):
    shutil.rmtree(dst_folder)

shutil.copytree(src_folder, dst_folder)
print(f"Copied {src_folder} to {dst_folder}")

Copied simplified_models to ../assets/simplified_objects/simplified_models


In [1]:
import pybullet as p, time
p.connect(p.GUI)                              # opens viewer
robot = p.loadURDF("../assets/simplified_objects/000.urdf",
                   useFixedBase=True)
while p.isConnected():
    p.stepSimulation()
    time.sleep(1/240.)


pybullet build time: Mar 18 2025 18:49:22
startThreads creating 1 threads.
starting thread 0
started thread 0 
argc=2
argv[0] = --unused
argv[1] = --start_demo_name=Physics Server
ExampleBrowserThreadFunc started
X11 functions dynamically loaded using dlopen/dlsym OK!
X11 functions dynamically loaded using dlopen/dlsym OK!
Creating context
Created GL 3.3 context
Direct GLX rendering context obtained
Making context current
GL_VENDOR=AMD
GL_RENDERER=AMD Radeon Graphics (radeonsi, renoir, ACO, DRM 3.61, 6.14.2-arch1-1)
GL_VERSION=4.6 (Core Profile) Mesa 25.0.3-arch1.1
GL_SHADING_LANGUAGE_VERSION=4.60
pthread_getconcurrency()=0
Version = 4.6 (Core Profile) Mesa 25.0.3-arch1.1
Vendor = AMD
Renderer = AMD Radeon Graphics (radeonsi, renoir, ACO, DRM 3.61, 6.14.2-arch1-1)
b3Printf: Selected demo: Physics Server
startThreads creating 1 threads.
starting thread 0
started thread 0 
MotionThreadFunc thread started
ven = AMD
ven = AMD
numActiveThreads = 0
stopping threads
Thread with taskId 0 exiti